In [2]:
# !pip install pandas
# !pip install nltk
!pip install rouge-score
!pip install bert-score
# !pip install scikit-learn


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=ed4430979fdff8b0265345d4c1aa95c27a58d7e6a55dda90aa031056d3ab8551
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  

In [6]:
!pip install nltk rouge-score bert-score


In [7]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def calculate_bleu(reference, candidate):
    references = [reference]  # BLEU requires a list of references
    candidate = candidate.split()
    references = [ref.split() for ref in references]
    return sentence_bleu(references, candidate, smoothing_function=SmoothingFunction().method1)


In [8]:
from rouge_score import rouge_scorer

def calculate_rouge(reference, candidate):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, candidate)
    return scores


In [9]:
from bert_score import score as bert_score

def calculate_bertscore(reference, candidate):
    P, R, F1 = bert_score([candidate], [reference], lang='en', return_hash=True)
    return P.mean().item(), R.mean().item(), F1.mean().item()


In [10]:
def calculate_mmr(query_embedding, document_embeddings, lambda_param=0.5):
    scores = []
    for doc_embedding in document_embeddings:
        relevance_score = np.dot(query_embedding, doc_embedding)
        diversity_score = np.max([np.dot(doc_embedding, other_doc) for other_doc in document_embeddings])
        mmr_score = lambda_param * relevance_score - (1 - lambda_param) * diversity_score
        scores.append(mmr_score)
    return scores


In [12]:
!pip install --upgrade bert-score

In [13]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from bert_score import score as bert_score

# Load the CSV file
df = pd.read_csv('easy.csv')

def calculate_bleu(reference, candidate):
    references = [reference]
    candidate = candidate.split()
    references = [ref.split() for ref in references]
    return sentence_bleu(references, candidate, smoothing_function=SmoothingFunction().method1)

def calculate_rouge(reference, candidate):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, candidate)
    return scores

def calculate_bertscore(reference, candidate):
    P, R, F1 = bert_score([candidate], [reference], lang='en', return_hash=False)
    return P.mean().item(), R.mean().item(), F1.mean().item()

bleu_scores = []
rouge_scores = []
bertscores = []

for index, row in df.iterrows():
    reference = row['answer']
    candidate = row['rag_answer']

    bleu = calculate_bleu(reference, candidate)
    bleu_scores.append(bleu)

    rouge = calculate_rouge(reference, candidate)
    rouge_scores.append(rouge)

    bert_p, bert_r, bert_f1 = calculate_bertscore(reference, candidate)
    bertscores.append((bert_p, bert_r, bert_f1))

metrics_df = pd.DataFrame({
    'BLEU Score': bleu_scores,
    'ROUGE Scores': rouge_scores,
    'BERTScore Precision': [x[0] for x in bertscores],
    'BERTScore Recall': [x[1] for x in bertscores],
    'BERTScore F1': [x[2] for x in bertscores]
})

metrics_df.to_csv('evaluation_metrics.csv', index=False)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro